In [1]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!make

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 15.52 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-variable-Wunused-variable]8;;]
  253 |                 int n = _wri

In [32]:
with open('nomes.classes', 'w') as f:
    f.write('zebra\n')
    f.write('gato\n')

In [3]:
!cat nomes.classes

zebra
gato


In [34]:
with open('obj.data', 'w') as f:
    f.write('classes = 2\n')
    f.write('train = sample_data/imagens_treino/train.txt\n')
    f.write('valid = sample_data/imagens_validacao/valid.txt\n')
    f.write('names = sample_data/nomes.classes\n')
    f.write('backup = backup/\n')

In [35]:
!cat obj.data

classes = 2
train = sample_data/imagens_treino/train.txt
valid = sample_data/imagens_validacao/valid.txt
names = sample_data/nomes.classes
backup = backup/


In [36]:
!cp cfg/yolov3.cfg cfg/yolov3_custom.cfg

In [7]:
with open('cfg/yolov3_custom.cfg', 'r+') as file:
  lines = file.readlines()

  for i, line in enumerate(lines):
        if "classes" in line:
            lines[i] = "classes = 2\n"  # Altere o número de classes conforme necessário

        if "filters" in line and "convolutional" in lines[i - 1]:
            lines[i] = "filters = 21\n"  # Altere o número de filtros conforme necessário

  file.seek(0)
  file.writelines(lines)
  file.truncate()  # Limpa o restante do arquivo se o novo conteúdo for menor que o original
  file.close()

In [37]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2024-03-08 18:19:15--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.2’

yolov3.weights.2    100%[===================>] 236.52M  45.1MB/s    in 5.3s    

2024-03-08 18:19:20 (44.3 MB/s) - ‘yolov3.weights.2’ saved [248007048/248007048]



In [38]:
# verificando se o arquivo yolov3.weights foi baixado
!ls

3rdparty	DarknetConfig.cmake.in	image_yolov3.sh        nomes.classes  vcpkg.json
backup		darknet_images.py	image_yolov4.sh        obj	      vcpkg.json.opencv23
build		darknet.py		include		       obj.data       video_yolov3.sh
build.ps1	darknet_video.py	json_mjpeg_streams.sh  package.xml    video_yolov4.sh
cfg		data			LICENSE		       README.md      yolov3.conv.81
cmake		docker-compose.yml	Makefile	       results	      yolov3.weights
CMakeLists.txt	Dockerfile.cpu		net_cam_v3.sh	       scripts	      yolov3.weights.1
darknet		Dockerfile.gpu		net_cam_v4.sh	       src	      yolov3.weights.2


In [39]:
# Iniciando o treinamento
!./darknet partial cfg/yolov3_custom.cfg yolov3.weights yolov3.conv.81 81

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x 128 0.001 BF
   9 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
  10 conv    128  

In [40]:
!apt-get install -y python3-opencv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-opencv is already the newest version (4.5.4+dfsg-9ubuntu4+jammy0).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [43]:
!./darknet detector train data/obj.data cfg/yolov3_custom.cfg yolov3.conv.81

 GPU isn't used 
 OpenCV isn't used - data augmentation will be slow 
yolov3_custom
mini_batch = 1, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     21       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  21 0.523 BF
   2 conv     21       1 x 1/ 1    208 x 208 x  21 ->  208 x 208 x  21 0.038 BF
   3 conv     21       3 x 3/ 1    208 x 208 x  21 ->  208 x 208 x  21 0.343 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  21 0.001 BF
   5 conv     21       3 x 3/ 2    208 x 208 x  21 ->  104 x 104 x  21 0.086 BF
   6 conv     21       1 x 1/ 1    104 x 104 x  21 ->  104 x 104 x  21 0.010 BF
   7 conv     21       3 x 3/ 1    104 x 104 x  21 ->  104 x 104 x  21 0.086 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 104 x  21 0.000 BF
   9 conv     21       1 x 1/ 1    104 x 104 x  21 ->  104 x 104 x  21 0.010 BF
  1